## PMEL ERDDAP M2 Data Visualization

__pyversion__==3.6   
__author__==S.Bell

In [1]:
import datetime
print("Last run {0}".format(datetime.datetime.now()))

Last run 2020-05-20 13:34:14.474987


In [2]:
import warnings
#remove the numpy/pandas/cython warnings
warnings.filterwarnings(action='ignore', message="numpy.dtype size changed,")

from erddapy import ERDDAP
import pandas as pd
import numpy as np

#server_url='http://downdraft.pmel.noaa.gov:8080/erddap'
server_url='http://ferret.pmel.noaa.gov/pmel/erddap'
e = ERDDAP(server=server_url)

df = pd.read_csv(e.get_search_url(response='csv', search_for='EcoFOCI'))
print(df['Dataset ID'].values)

['EcoFOCI_Bering_Sea_timeseries_data' 'ChukchiSea_454a_037a_fcf4'
 'arcticRescueData_e5bf_cef8_3f72' 'Shelikof_line8_3695_0ada_d066'
 'dy1104_profile_data' 'dy1408_profile_data' 'mf0904_profile_data']


In [3]:
dataset_id = 'EcoFOCI_Bering_Sea_timeseries_data'

### retrieving and plotting data

In [4]:
kw = {
    'standard_name': 'sea_water_temperature',
    'min_lon': -180.0,
    'max_lon': -130.0,
    'min_lat': 55.0,
    'max_lat': 90.0,
    'min_time': '2010-01-10T00:00:00Z',
    'max_time': '2019-01-10T00:00:00Z',
}

constraints = {
    'longitude>=': kw['min_lon'],
    'longitude<=': kw['max_lon'],
    'latitude>=': kw['min_lat'],
    'latitude<=': kw['max_lat'],
    'time>=': kw['min_time'],
    'time<=': kw['max_time'],
}

variables =  e.get_var_by_attr(dataset_id=dataset_id, standard_name=lambda v: v is not 'NC_GLOBAL')
while 'NC_GLOBAL' in variables: variables.remove('NC_GLOBAL')
    
download_url = e.get_download_url(
    dataset_id=dataset_id,
    protocol='tabledap',
    response='csv',
    variables=variables,
)

print(download_url)

http://ferret.pmel.noaa.gov/pmel/erddap/tabledap/EcoFOCI_Bering_Sea_timeseries_data.csv?photosynthetically_active_radiation,ocean_depth,id,ocean_practical_salinity_1,ocean_velocity_v_northward_true_north,ocean_chlorophyll_fluorescence_raw_standard_deviation,ocean_conductivity_1,latitude,photosynthetically_active_radiation_raw,ocean_turbidity_FNU,ocean_temperature_1,ocean_dissolved_oxygen_concentration_1_mMperkg,ocean_velocity_u_eastward_true_north,time,ocean_current_direction_component_true_north,ocean_chlorophyll_a_concentration_factoryCal,ocean_current_speed_component,lon360,longitude,ocean_oxygen_saturation_1,depth,ocean_chlorophyll_fluorescence_raw


In [5]:
from requests.exceptions import HTTPError

dfs = {}
try:
    e = ERDDAP(server=server_url,
        protocol='tabledap',
        response='csv'
    )
    e.constraints=constraints
    e.variables=variables
    e.dataset_id=dataset_id
except HTTPError:
    print('Failed to generate url {}'.format(dataset_id))

try:
    dfs.update({dataset_id: e.to_pandas(
                        index_col='time',
                        parse_dates=True,
                        skiprows=(1,)  # units information can be dropped.
                        )})
except:
    pass

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cmocean

for dfn, df in dfs.items():
    fig, ax = plt.subplots(figsize=(17, 2))
    cs = ax.scatter(df.index, df['depth'], s=15, c=df['ocean_temperature_1'], marker='o', edgecolor='none', vmin=-2, vmax=10, cmap=cmocean.cm.thermal)

    ax.invert_yaxis()
    ax.set_xlim(df.index[0], df.index[-1])
    xfmt = mdates.DateFormatter('%d-%b')
    ax.xaxis.set_major_formatter(xfmt)

    cbar = fig.colorbar(cs, orientation='vertical', extend='both')
    cbar.ax.set_ylabel('Temperature ($^\circ$C)')
    ax.set_ylabel('Depth (m)')
    ax.set_xlabel(dfn)
    
    #fig.savefig(dfn+'.png',dpi=300)

In [10]:
for dfn, df in dfs.items():
    fig, ax = plt.subplots(figsize=(17, 2))
    cs = ax.scatter(df.index, df['depth'], s=15, c=df['ocean_practical_salinity_1'], marker='o', edgecolor='none', vmin=31, vmax=33, cmap=cmocean.cm.haline)

    ax.invert_yaxis()
    ax.set_xlim(df.index[0], df.index[-1])
    xfmt = mdates.DateFormatter('%d-%b')
    ax.xaxis.set_major_formatter(xfmt)

    cbar = fig.colorbar(cs, orientation='vertical', extend='both')
    cbar.ax.set_ylabel('Salinity (PSU)')
    ax.set_ylabel('Depth (m)')
    ax.set_xlabel(dfn)
    
    #fig.savefig(dfn+'.png',dpi=300)

In [11]:
for dfn, df in dfs.items():
    fig, ax = plt.subplots(figsize=(17, 2))
    cs = ax.scatter(df.index, df['depth'], s=15, c=df['ocean_chlorophyll_a_concentration_factoryCal'], marker='o', edgecolor='none', vmin=0, vmax=15, cmap=cmocean.cm.algae)

    ax.invert_yaxis()
    ax.set_xlim(df.index[0], df.index[-1])
    xfmt = mdates.DateFormatter('%d-%b')
    ax.xaxis.set_major_formatter(xfmt)

    cbar = fig.colorbar(cs, orientation='vertical', extend='both')
    cbar.ax.set_ylabel('Salinity (PSU)')
    ax.set_ylabel('Depth (m)')
    ax.set_xlabel(dfn)
    
    #fig.savefig(dfn+'.png',dpi=300)